In [1]:
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import seaborn as sns
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import networkx as nx
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import adfuller, acf, q_stat
import sys
sys.path.append('../src')
from models import arima_model, plot_arima_results, plot_pacf_for_arima

In [2]:
us_cpi_df = pd.read_excel(r"../data/raw/revised-seasonally-adjusted-indexes-2021.xlsx")
excel_df_16_20 = pd.read_excel(r"../data/raw/revised-seasonally-adjusted-indexes-2020.xlsx")
excel_df_15_19 = pd.read_excel(r"../data/raw/revised-seasonally-adjusted-indexes-2019.xlsx")

#we are only keeping the SAESONALLY ADJUSTED INDEX values so it will be dropped to reduce dataset clutter for pivoting
# Also dropping "ITEM" because it provides a code for the item but we will just be mapping based off title, will make the pivoted dataframe easier to read
us_cpi_df = us_cpi_df[(us_cpi_df['DATA_TYPE'] == 'SEASONALLY ADJUSTED INDEX')]

excel_df = pd.concat([excel_df_16_20, excel_df_15_19[excel_df_15_19['YEAR'] == 2015]], ignore_index=True).sort_values(by='YEAR')
us_cpi_df = excel_df[excel_df['DATA_TYPE'] == 'SEASONALLY ADJUSTED INDEX'].drop(columns='DATA_TYPE')
us_cpi_df[us_cpi_df['TITLE'] == 'All items'].sort_values(by='YEAR')

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,ITEM,TITLE,seriesid,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
6450,SA0,All items,CUSR0000SA0,2015,234.747,235.342,235.976,236.222,237.001,237.657,238.034,238.033,237.498,237.733,238.017,237.761
7200,SA0,All items,CWSR0000SA0,2015,229.498,230.264,230.948,231.118,232.035,232.720,233.021,232.967,232.092,232.262,232.491,232.088
3465,SA0,All items,CWSR0000SA0,2016,231.862,231.279,232.202,233.163,233.740,234.419,234.117,234.480,235.193,235.776,235.992,236.654
0,SA0,All items,CUSR0000SA0,2016,237.652,237.336,238.080,238.992,239.557,240.222,240.101,240.545,241.176,241.741,242.026,242.637
1,SA0,All items,CUSR0000SA0,2017,243.620,243.872,243.766,244.274,244.069,244.218,244.280,245.205,246.551,246.657,247.378,247.736
3466,SA0,All items,CWSR0000SA0,2017,237.722,237.850,237.678,238.174,237.901,238.017,237.995,238.994,240.611,240.611,241.486,241.814
2,SA0,All items,CUSR0000SA0,2018,248.721,249.300,249.517,250.275,250.786,251.152,251.345,251.735,252.183,252.899,252.822,252.493
3467,SA0,All items,CWSR0000SA0,2018,242.885,243.428,243.510,244.330,244.847,245.262,245.364,245.782,246.233,247.094,246.825,246.124
3468,SA0,All items,CWSR0000SA0,2019,245.961,246.527,247.807,249.098,248.995,248.930,249.475,249.562,249.958,250.971,251.514,251.791
3,SA0,All items,CUSR0000SA0,2019,252.441,252.969,254.147,255.326,255.371,255.423,255.925,256.118,256.532,257.387,257.989,258.203


In [3]:
df_melted = us_cpi_df.melt(id_vars=['ITEM', 'seriesid', 'TITLE', 'YEAR'],
                     var_name='Month',
                     value_name='Value')

df_melted['Date'] = pd.to_datetime(df_melted['YEAR'].astype(str) + '-' + df_melted['Month'], format='%Y-%b')

# Drop unnecessary columns and sort by Date
df_melted = df_melted.drop(columns=['YEAR', 'Month']).sort_values(by='Date')

# Show the filtered result for 'All items'
df_melted_all_items = df_melted[df_melted['TITLE'] == 'All items']
df_melted_all_items

,ITEM,seriesid,TITLE,Value,Date
405,SA0,CWSR0000SA0,All items,229.498,2015-01-01
153,SA0,CUSR0000SA0,All items,234.747,2015-01-01
2770,SA0,CUSR0000SA0,All items,235.342,2015-02-01
3022,SA0,CWSR0000SA0,All items,230.264,2015-02-01
5639,SA0,CWSR0000SA0,All items,230.948,2015-03-01
...,...,...,...,...,...
25836,SA0,CUSR0000SA0,All items,260.462,2020-10-01
28585,SA0,CWSR0000SA0,All items,254.589,2020-11-01
28453,SA0,CUSR0000SA0,All items,260.927,2020-11-01
31202,SA0,CWSR0000SA0,All items,255.275,2020-12-01


In [4]:
df_avg = df_melted.groupby(['Date', 'TITLE']).agg({'Value': 'mean'}).reset_index()

In [5]:
# Pivot the table to make 'TITLE' columns
df_pivoted = df_melted.pivot_table(index='Date', columns='TITLE', values='Value')

# Display the transformed DataFrame
df_pivoted.head()

TITLE,Admissions,Airline fares,Alcoholic beverages,Alcoholic beverages at home,All items,All items less energy,All items less food,All items less food and energy,All items less food and shelter,"All items less food, shelter, and energy",...,Water and sewer and trash collection services,Water and sewerage maintenance,Wine at home,Women's and girls' apparel,Women's apparel,Women's dresses,Women's footwear,Women's outerwear,Women's suits and separates,"Women's underwear, nightwear, swimwear, and accessories"
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,341.3555,294.0830,239.6795,200.0960,232.1225,237.1845,229.7640,236.1250,215.378,220.496,...,210.3445,478.1760,166.9035,111.6825,114.5565,120.7260,128.4795,114.6895,81.9765,105.5945
2015-02-01,346.1580,292.8945,239.8730,200.1150,232.8030,237.5805,230.5125,236.5475,215.995,220.657,...,211.2700,480.3765,167.4480,110.8755,113.4480,120.0275,128.9505,112.5570,81.9900,105.0315
2015-03-01,350.2950,289.0925,240.2010,200.3050,233.4620,237.9940,231.3390,237.1080,216.811,221.123,...,211.7550,481.7970,167.6920,110.9035,113.8615,120.8925,130.0060,109.8650,82.2275,106.0315
2015-04-01,350.5850,284.5065,240.2720,200.1965,233.6700,238.4660,231.5960,237.6860,216.891,221.626,...,212.3015,483.1880,167.2005,111.2305,114.2740,118.4830,129.6240,110.7850,83.6730,105.3500
2015-05-01,354.4515,293.4700,240.9935,200.6975,234.5180,238.7845,232.5430,238.0215,217.934,221.866,...,213.0505,484.8855,167.3010,110.5240,113.6050,116.7380,126.7700,109.8115,83.3920,105.0775


In [6]:
categories_mapping = us_data = {
    'Shelter': {
        'Electricity': 'Owned accommodation',
        'Fuels and utilities': 'Owned accommodation',
        'Household energy': 'Owned accommodation',
        'Housing': 'Owned accommodation',
        'Housing at school, excluding board': 'Rented accommodation',
        "Owners' equivalent rent of primary residence": 'Owned accommodation',
        "Owners' equivalent rent of residences": 'Owned accommodation',
        'Rent of primary residence': 'Rented accommodation',
        'Rent of shelter': 'Rented accommodation',
        'Services less rent of shelter': 'Rented accommodation',
        'Utility (piped) gas service': 'Water, fuel and electricity',
        'Water and sewer and trash collection services': 'Water, fuel and electricity',
        'Water and sewerage maintenance': 'Water, fuel and electricity'
    },
    'Clothing and footwear': {
        'Apparel': 'Clothing',
        'Apparel less footwear': 'Clothing',
        "Boys' and girls' footwear": 'Footwear',
        "Boys' apparel": 'Clothing',
        'Footwear': 'Footwear',
        "Girls' apparel": 'Clothing',
        "Infants' and toddlers' apparel": 'Clothing',
        'Jewelry': 'Clothing accessories, watches and jewellery',
        'Jewelry and watches': 'Clothing accessories, watches and jewellery',
        "Men's and boys' apparel": 'Clothing',
        "Men's apparel": 'Clothing',
        "Men's pants and shorts": 'Clothing',
        "Men's shirts and sweaters": 'Clothing',
        "Men's suits, sport coats, and outerwear": 'Clothing',
        "Women's and girls' apparel": 'Clothing',
        "Women's apparel": 'Clothing',
        "Women's dresses": 'Clothing',
        "Women's footwear": 'Footwear',
        "Women's outerwear": 'Clothing',
        "Women's suits and separates": 'Clothing',
        "Women's underwear, nightwear, swimwear, and accessories": 'Clothing'
    }


}


In [7]:
grouped_data = {}

for category, subcategories in categories_mapping.items():
    for column, group in subcategories.items():
        if column in df_pivoted.columns:  # Ensure the column exists in the dataset
            if group not in grouped_data:
                grouped_data[group] = []
            grouped_data[group].append(df_pivoted[column])

# Compute the mean for each group
shelter_clothing_df = pd.DataFrame({group: pd.concat(cols, axis=1).mean(axis=1) for group, cols in grouped_data.items()})

shelter_columns = [col for col, group in categories_mapping['Shelter'].items() if col in df_pivoted.columns]
clothing_columns = [col for col, group in categories_mapping['Clothing and footwear'].items() if col in df_pivoted.columns]

shelter_clothing_df['Shelter'] = shelter_clothing_df[['Owned accommodation', 'Rented accommodation', 'Water, fuel and electricity']].mean(axis=1)
shelter_clothing_df['Clothing and Footwear'] = shelter_clothing_df[['Clothing',	'Footwear',	'Clothing accessories, watches and jewellery']].mean(axis=1)
shelter_clothing_df

,Owned accommodation,Rented accommodation,"Water, fuel and electricity",Clothing,Footwear,"Clothing accessories, watches and jewellery",Shelter,Clothing and Footwear
Date,,,,,,,,
2015-01-01,235.330417,338.392250,288.530000,110.519687,139.103500,165.08600,287.417556,138.236396
2015-02-01,235.639000,339.240875,288.297333,110.513156,139.107333,163.33175,287.725736,137.650747
2015-03-01,235.251583,339.890250,287.897667,111.193750,140.407333,163.18425,287.679833,138.261778
2015-04-01,235.215250,340.701125,287.042500,110.761594,139.673167,162.65775,287.652958,137.697503
2015-05-01,234.646583,341.320875,287.452167,110.312188,139.260167,162.07875,287.806542,137.217035
...,...,...,...,...,...,...,...,...
2020-08-01,260.068083,393.703750,334.911667,100.024906,134.257500,163.97425,329.561167,132.752219
2020-09-01,261.274000,394.423625,337.300333,99.067531,132.058000,167.33275,330.999319,132.819427
2020-10-01,262.070000,394.921375,338.044167,97.751969,134.516833,166.07300,331.678514,132.780601


In [8]:
for column in df_pivoted.columns:
    print(column)


Admissions
Airline fares
Alcoholic beverages
Alcoholic beverages at home
All items
All items less energy
All items less food
All items less food and energy
All items less food and shelter
All items less food, shelter, and energy
All items less food, shelter, energy, and used cars and trucks
All items less medical care
All items less shelter
Apparel
Apparel less footwear
Apples
Appliances
Bacon and related products
Bacon, breakfast sausage, and related products
Beef and veal
Beer, ale, and other malt beverages at home
Beverage materials including coffee and tea
Boys' and girls' footwear
Boys' apparel
Breakfast sausage and related products
Butter
Butter and margarine
Cable and satellite television service
Cakes, cupcakes, and cookies
Canned fruits
Canned fruits and vegetables
Canned vegetables
Car and truck rental
Carbonated drinks
Cereals and bakery products
Cereals and cereal products
Cigarettes
Citrus fruits
Coffee
College tuition and fees
Commodities
Commodities less food
Commodities

In [9]:
df_model_data_USA = pd.read_csv('../data/processed/US_Categorized_Products_and_Services_NEW.csv')

dict_USA = df_model_data_USA.drop_duplicates(subset=['Product_Service']).set_index('Product_Service')['Category'].to_dict()
df_pivoted.reset_index(inplace=True)
df_USA_CPI_melt = pd.melt(df_pivoted, var_name = 'Products and product groups', value_name = 'VALUE', id_vars = 'Date')

df_USA_CPI_melt['Category'] = df_USA_CPI_melt['Products and product groups'].map(dict_USA)

In [10]:
df_USA_CPI_pivot = df_USA_CPI_melt.pivot_table(index = 'Date', columns = 'Category', values = 'VALUE')

In [11]:
_, _, forecast_df, post_tariff_data, tariff_date, model = arima_model(df_USA_CPI_pivot, 'Groceries', (12,1,3), '2018-06-01', 8, 8)



📊 **Augmented Dickey-Fuller (ADF) Test on Differenced Data:**
  p-value: 0.1125 (Non-Stationary ❌)

📊 **In-Sample Evaluation Metrics**
    MAE = 0.2636, MAEP = 0.12%
    RMSE = 0.3111

**Ljung-Box Test (Residuals Autocorrelation):**
  p-value: 0.1657 (No autocorrelation ✅)

📊 **Breusch-Pagan Test (Heteroskedasticity):**
  p-value: 0.1281 (Homoskedastic ✅)

📊 **Model Selection Criteria:**
  AIC: 70.8082


/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [12]:
forecast_df.reset_index(inplace=True)
post_tariff_data.reset_index(inplace=True)
df_USA_CPI_pivot.reset_index(inplace=True)
chart1 = alt.Chart(forecast_df).mark_line(color='blue').encode(x = 'index:N', y = alt.Y('Forecast:Q', scale = alt.Scale(domain = [220, 283])))

chart2 = alt.Chart(post_tariff_data).mark_line(color='red').encode(x = 'index:N', y = alt.Y('Actuals:Q', scale = alt.Scale(domain = [220, 283])))
chart3 = alt.Chart(df_USA_CPI_pivot).mark_line(color='black').encode(x = 'Date:N', y = alt.Y('Groceries:Q', scale = alt.Scale(domain = [220, 289])))
# chart4 = alt.Chart(df_future).mark_line(color='green').encode(x = 'Date:N', y = alt.Y('Forecast:Q', scale = alt.Scale(domain = [50, 149])))

chart1+chart2+chart3

alt.LayerChart(...)